- 실습 중인 쇼핑몰 상품은 여러 페이지에 걸쳐 진열되어 있다.
    - 함수 구성 후 여러 페이지 크롤링
    - 에러 발생하는 부분은 함수 수정
    - 함수를 기능별로 세분화

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import ssl

In [2]:
# 사이트 url
url = 'http://jolse.com/category/toners-mists/1019/'

### 함수를 기능별로 세분화
- 요청 및 파싱 객체 생성 : get_request_product(url), 반환값 : bs4 객체
- 전체 정보 추출 : get_page_product(bs_obj)
    - bs_obj를 전달받아서
    - get_proudct_info(box) 호출 (제품 1개에 대한 정보 반환)
    - 1페이지 제품에 대한 df을 생성 후 반환
- 각 제품의 정보 추출 : get_proudct_info(box)
    - 제품정보 1개에 대해 제품명/가격/세일가격을 추출해서 반환

In [3]:
# df 생성
prd_dict = {'품목':[], '가격':[], '세일가격':[]}
prd_df = pd.DataFrame(prd_dict)
prd_df

,품목,가격,세일가격


In [4]:
# 1. 요청 및 파싱객체(bs4) 생성 함수
def get_request_product(url) :
    
    # 요청 후 코드 추출
    url = url
    context = ssl._create_unverified_context()
    htmls = urlopen(url, context=context)
    htmls = htmls.read()
    
    # bs4 객체 생성
    bs_obj = BeautifulSoup(htmls,'html.parser')
    
    return bs_obj

In [5]:
# 전체 정보 추출 : get_page_product(bs_obj)
# bs_obj를 전달받아서 get_proudct_info(box)를 호출 (제품 1개에 대한 정보 반환)
# 전역변수 df 활용해서 제품 정보 저장
# 최초 호출함수
def get_page_product(url) :
    
    # global 변수를 함수 내부가 아닌 외부에서 찾을 것
    # global은 전역변수(함수내/외부에서 모두 사용)로 생성
    global prd_df 
    bs_obj = get_request_product(url)
    
    # 페이지 내에 전체 제품 정보 추출
    boxes = bs_obj.findAll('div',{'class':'description'})
    
    # 각 제품에 대한 정보 추출 후 df에 저장
    for box in boxes[2:] :  # recommendations 제외
        res = pd.DataFrame(get_product_info(box), index=range(1,2))
        prd_df = pd.concat([prd_df,res], axis=0, ignore_index=True)

In [6]:
# 각 제품 정보 추출하는 함수
# box는 class 값이 description인 div 태그 1개가 전달
def get_product_info(box) :
    try : 
        title=box.find('strong',{'class':'name'}).text.split(":")[1].strip()
        lis = box.find('ul').findAll('li')
        price=(lis[0].text.split(' ')[2])
        sale_price=(lis[1].text.split(' ')[2])
        return{'품목': title, '가격': price, '세일가격': sale_price}
    except : 
        print('에러')

In [7]:
url

'http://jolse.com/category/toners-mists/1019/'

In [8]:
# 함수 호출 테스트
get_page_product(url)

In [9]:
prd_df

,품목,가격,세일가격
0,SOME BY MI AHA BHA PHA 30 Days Miracle Toner 1...,24.00,15.89
1,COSRX BHA BLACKHEAD POWER LIQUID 100 ML,20.81,13.49
2,COSRX AHA/BHA CLARIFYING TREATMENT TONER 150ml,17.25,10.79
3,Pyunkang Yul Essence Toner 200ml,21.90,13.49
4,Beauty of Joseon Ginseng Essence Water 150ml,24.55,14.79
5,ROUND LAB 1025 Dokdo Toner 200ml,17.00,13.59
6,COSRX Full Fit Propolis Synergy Toner 150ml,28.00,12.49
7,Isntree Green tea Fresh Toner 200ml,19.90,15.92
8,Isntree Hyaluronic Acid Toner 200ml (Renewal),19.30,14.24
9,COSRX AHA 7 WHITEHEAD POWER LIQUID 100ml,17.81,10.69


### 여러 페이지 추출
- 해당 쇼핑몰의 url 분석
- 각 페이지에 대하여 url에 파라미터가 다르게 전송됨
- https://jolse.com/category/toners-mists/1019?page= + 페이지번호
- url 작성시 페이지번호를 따로 부착

In [10]:
base_url = 'http://jolse.com/category/toners-mists/1019?page='

# 2페이지부터 4페이지 까지 제품 추출
for i in range(2,5) :
    url = base_url + str(i)
    get_page_product(url)

In [11]:
prd_df

,품목,가격,세일가격
0,SOME BY MI AHA BHA PHA 30 Days Miracle Toner 1...,24.00,15.89
1,COSRX BHA BLACKHEAD POWER LIQUID 100 ML,20.81,13.49
2,COSRX AHA/BHA CLARIFYING TREATMENT TONER 150ml,17.25,10.79
3,Pyunkang Yul Essence Toner 200ml,21.90,13.49
4,Beauty of Joseon Ginseng Essence Water 150ml,24.55,14.79
...,...,...,...
75,MISSHA Glow Skin Balm To Go Mist 80ml,21.48,19.33
76,THANK YOU FARMER Rice Pure Essential Toner 200ml,28.00,25.20
77,HOLIKA HOLIKA Good Cera Super Ceramide Mist 120ml,18.76,16.88
78,Huxley TONER EXTRACT IT 120ml,32.00,28.80


In [12]:
# 수집데이터 저장
prd_df.to_csv('./crawl_data/beauty_price.csv')

## 연습문제
1. recommendations 제품을 슬라이싱이 아닌 방법으로 recommendations 제품의 진열대(ul 태그)를 제외하고
2. 모든 제품 진열대(ul 태그)를 이용해서 제품을 추출하도록 함수 수정
3. 전체 페이지가 몇 페이지인지를 확인하는 크롤링 코드를 작성해서 전체 페이지를 얻어오기
    - 3번에서 얻어온 전체 페이지 수 만큼 크롤링 되도록 수정

#### 연습문제 작성 후 제출 폴더에 현재 파일 제출 (파일 제목 변경 후 제출)

In [13]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import ssl

In [14]:
# df 생성
prd_dict = {'품목':[], '가격':[], '세일가격':[]}
prd_df = pd.DataFrame(prd_dict)
prd_df

,품목,가격,세일가격


In [15]:
# 요청 및 파싱객체(bs4) 생성 함수
def get_request_product(url) :
    
    # 요청 후 코드 추출
    url = url
    context = ssl._create_unverified_context()
    htmls = urlopen(url, context=context)
    htmls = htmls.read()
    
    # bs4 객체 생성
    bs_obj = BeautifulSoup(htmls,'html.parser')
    
    return bs_obj

In [16]:
# 호출 시작 함수
def get_page_product(url) :
    
    # global 변수를 함수 내부가 아닌 외부에서 찾을 것
    # global은 전역변수(함수내/외부에서 모두 사용)로 생성
    global prd_df 
    bs_obj = get_request_product(url)
    
    # 페이지 내에 전체 제품 정보 추출
    page = bs_obj.findAll('ul', {'class': 'prdList grid5'})[1]
    boxes = page.findAll('div', {'class':'description'})
    
    # 각 제품에 대한 정보 추출 후 df에 저장
    for box in boxes : 
        res = pd.DataFrame(get_product_info(box), index=range(1,2))
        prd_df = pd.concat([prd_df,res], axis=0, ignore_index=True)

In [17]:
# 각 제품 정보 추출하는 함수
def get_product_info(box) :
    try : 
        title=box.find('strong', {'class':'name'}).text.split(":")[1].strip()
        lis = box.find('ul').findAll('li')
        price=(lis[0].text.split(' ')[2])
        sale_price=(lis[1].text.split(' ')[2])
        return{'품목': title, '가격': price, '세일가격': sale_price} 
    except : 
        print('에러')

In [18]:
url = 'http://jolse.com/category/toners-mists/1019?page=1'

In [19]:
# 함수 호출 테스트
# prd_df에 전달된 url 페이지의 일반 진열 제품의 정보를 저장해주는 함수
get_page_product(url)

In [20]:
prd_df  # 18개 상품 (recommendations 제외한 상품)

,품목,가격,세일가격
0,SOME BY MI AHA BHA PHA 30 Days Miracle Toner 1...,24.00,15.89
1,COSRX BHA BLACKHEAD POWER LIQUID 100 ML,20.81,13.49
2,COSRX AHA/BHA CLARIFYING TREATMENT TONER 150ml,17.25,10.79
3,Pyunkang Yul Essence Toner 200ml,21.90,13.49
4,Beauty of Joseon Ginseng Essence Water 150ml,24.55,14.79
5,ROUND LAB 1025 Dokdo Toner 200ml,17.00,13.59
6,COSRX Full Fit Propolis Synergy Toner 150ml,28.00,12.49
7,Isntree Green tea Fresh Toner 200ml,19.90,15.92
8,Isntree Hyaluronic Acid Toner 200ml (Renewal),19.30,14.24
9,COSRX AHA 7 WHITEHEAD POWER LIQUID 100ml,17.81,10.69


In [21]:
# 마지막 페이지 추출
# 요청 후 응답 객체의 코드를 bs4 객체로 생성
url
bs_obj = get_request_product(url)

In [22]:
last = int(bs_obj.find('a', {'class': 'last'})['href'].split('=')[1])
last

25

In [23]:
base_url = 'http://jolse.com/category/toners-mists/1019?page='

for i in range(1, last+1) :
    url = base_url + str(i)
#     print(url)
    get_page_product(url)

에러
에러
에러
에러
에러
에러
에러
에러
에러
에러
에러


- 수집할 때 수집되다가 특정 지점에서 에러 발생시 try ~ except 구문 활용해서 skip
    - 대량 데이터인 경우 1~2개 수집 에러가 문제 되지 않음